In [ ]:
import os
import math
import cv2
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

from src.image_processing import ImageProcessor
from src.tiles_segmentation import TileExtractor
from src.tiles_classification import TileClassifier

In [ ]:
def show(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots(1, 1)
    if title is not None:
        fig.suptitle(title)
    ax.imshow(image, cmap='gray')
    ax.set_axis_off()

In [ ]:
image = cv2.imread('./images/Pict_4_1.bmp')
#image = cv2.cvtColor(image_orig, cv2.COLOR_BGR2GRAY)
#image = cv2.GaussianBlur(image, (5, 5), 0)
show(image)

In [ ]:
thres_image_otsu = proc.process_image(image, 'otsu')
thres_image_ada = proc.process_image(image, 'adaptive')

contours_otsu, _ = cv2.findContours(thres_image_otsu,
                                    cv2.RETR_LIST,
                                    cv2.CHAIN_APPROX_SIMPLE)
contours_ada, _ = cv2.findContours(thres_image_ada,
                                   cv2.RETR_LIST,
                                   cv2.CHAIN_APPROX_SIMPLE)

contours_otsu = proc.process_contours(contours_otsu)
contours_ada = proc.process_contours(contours_ada)

unique_contours = proc.find_unique_contours(contours_otsu + contours_ada)

output_image = image.copy()
cv2.drawContours(output_image, [c.vertices for c in unique_contours], -1, (0, 255, 0), 2)

show(output_image)

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(12, 12))
ax = iter(ax.flatten())

for img_path in os.listdir('images'):
    if img_path.startswith('Pict'):
        image = cv2.imread(os.path.join('images/', img_path))
        assert image is not None, 'bad image'
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        axes = next(ax)

        proc = TileExtractor(area_min=2000,
                              min_dist_between_centr=20,
                              approx_thres=0.05,
                              equilaterial_thres=0.15)
        contr = proc.find_objects(image)
        output_image = image.copy()
        cv2.drawContours(output_image, [c.vertices for c in contr], -1, (0, 255, 0), 2)
        show(output_image, axes)
plt.tight_layout()
plt.show()

In [ ]:
proc = ImageProcessor()
proc.process_image('./images/Pict_2_2.bmp', 'Pict_2_2.info')

In [ ]:
image = cv2.imread('./images/Pict_2_2.bmp')
show(image)

classifier = TileClassifier(center_dist_thres=4,
                           min_dot_r=2.5,
                           max_dot_r=6)
ext = TileExtractor(area_min=2000,
                      min_dist_between_centr=20,
                      approx_thres=0.05,
                      equilaterial_thres=0.15)
contours = ext.find_objects(image)

for cropped in proc.extract_tile_images(image, contours):
    show(cropped, title=str(classifier.predict(cropped)) + f' {np.mean(cropped)}')
    #print(classifier.predict(cropped))

In [ ]:
centr_clrs = []

In [ ]:
#img = cropped[size // 2:, size // 2:]
def classify(img):
    chs_bgr = list(cv2.split(img))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    chs = list(cv2.split(img))
    chs.append(255 - chs[1])
    # chs += [255 - ch for ch in chs]
    # chs += chs_bgr
    # chs += [255 - ch for ch in chs_bgr]
    #chs = [cv2.split(img)[1]]
    thresh = np.zeros_like(chs[0])
    for ch in chs:
        ch = cv2.medianBlur(ch, 3)
        #ch = cv2.GaussianBlur(ch, (3, 3), 0)

        # se=cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        # bg=cv2.morphologyEx(ch, cv2.MORPH_DILATE, se)
        # ch=cv2.divide(ch, bg, scale=255)
        #show(ch)

        # t = cv2.adaptiveThreshold(ch, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
        #                                cv2.THRESH_BINARY_INV, 3, 2)
        #_, t = cv2.threshold(ch, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        #t = cv2.erode(t, (2, 2), iterations=2)
        #t = cv2.dilate(t, (3, 3), iterations=2)
        #t = cv2.morphologyEx(t, cv2.MORPH_OPEN, (3, 3), iterations=1)
        #show(ch)
        t = cv2.Canny(ch, 70, 110)
        #show(t)
        thresh = cv2.bitwise_or(thresh, t)
    show(thresh)

    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    output_mask = np.zeros_like(img)
    segmentated_dots = []
    for contour in contours:
        r = np.random.randint(50, 150)
        g = np.random.randint(50, 255 - r)
        b = 255 - r - g
        (x,y),r = cv2.minEnclosingCircle(contour)
        center = (int(x),int(y))
        dsts = [math.sqrt((center[0] - pt[0])**2 + (center[1] - pt[1])**2) for pt in segmentated_dots]
        if 2.5 < r < 6 and (len(dsts) == 0 or min(dsts) > 4):
            #centr_clrs.append(img[int(y), int(x)])
            cv2.drawContours(output_mask, [contour], -1, (r, g, b), 2)
            segmentated_dots.append(center)
    return output_mask

In [ ]:
def try_guess(image, guess) -> bool:
    channels_by_guess = {
        1: [7, 10],
        2: [1, 2, 4, 8],
        3: [2, 5],
        4: [1, 2, 4, 5],
        5: [7, 10],
    }

    chs_bgr = list(cv2.split(image))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    chs = list(cv2.split(image))
    chs += [255 - ch for ch in chs]
    chs += chs_bgr
    chs += [255 - ch for ch in chs_bgr]

    chs = np.array(chs)[channels_by_guess[guess]]
    thresh = np.zeros_like(chs[0])

    for ch in chs:
        ch = cv2.medianBlur(ch, 3)
        t = cv2.Canny(ch, 70, 120)
        thresh = cv2.bitwise_or(thresh, t)

    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    segmentated_dots = []
    for contour in contours:
        (x, y), r = cv2.minEnclosingCircle(contour)
        center = (int(x), int(y))
        dsts = [math.sqrt((center[0] - pt[0])**2 + (center[1] - pt[1])**2) for pt in segmentated_dots]
        if 1 < r < 7 and (len(dsts) == 0 or min(dsts) > 5):
            segmentated_dots.append(center)

    if guess == 5:
        show(thresh, title=f'guess={guess}, found={len(segmentated_dots)}')
    return len(segmentated_dots) == guess

In [ ]:
image = cv2.imread('./images/Pict_2_2.bmp')
image = (image / 255)
image[image > 1] = 1
image = (image * 255).astype(np.uint8)

proc = ImageProcessor()
ext = TileExtractor(area_min=2000,
                      min_dist_between_centr=20,
                      approx_thres=0.05,
                      equilaterial_thres=0.15)
contours = ext.find_objects(image)

for cropped in proc.extract_tile_images(image, contours):
    size = cropped.shape[0]
    clusters = [cropped[:size // 2, size // 4:size - size//4],
                cropped[size // 2:, :size // 2],
                cropped[size // 2:, size // 2:]]
    for c in clusters:
        ans = 0
        for i in range(1, 6):
            if try_guess(c, i):
                ans = i
                break
        show(c, title=ans)

In [ ]:
image = cv2.imread('./images/Pict_1_1.bmp')
# image = (image / 255) * 1.3
# image[image > 1] = 1
# image = (image * 255).astype(np.uint8)

proc = TileExtractor(area_min=2000,
                      min_dist_between_centr=20,
                      approx_thres=0.05,
                      equilaterial_thres=0.15)
contr = proc.find_objects(image)
for c in contr[1:]:
    # find <<bottom>> side of the triangle
    a, b = sorted(c.vertices[:, 0], key=lambda x: x[1], reverse=True)[:2]
    x, y = a - b

    # rotate image so the triangle will stand upright
    angle = np.degrees(np.arctan2(y, x))
    angle = angle if angle < 90 else angle + 180
    center = (c.centroid.x, c.centroid.y)

    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    rot = cv2.transform(c.vertices, M)

    x, y, w, h = cv2.boundingRect(rot)
    size = max(w, h)
    cropped = rotated[y:y + size, x:x + size]
    clusters = [cropped[:size // 2, size // 4:size - size//4],
                cropped[size // 2:, :size // 2],
                cropped[size // 2:, size // 2:]]
    show(cropped)
    for c in clusters:
        print(np.mean(c))
        show(classify(c))
    break

In [ ]:
clrs = np.array(centr_clrs)
clrs.shape

In [ ]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=6, metric='cosine')
model.fit(clrs)

In [ ]:
model.cluster_centers_

In [ ]:
from sklearn.manifold import TSNE

embed = TSNE(perplexity=10).fit_transform(clrs)

fig, ax = plt.subplots(1, 2)

ax[0].scatter(*embed.T, c=clrs / 255)
ax[0].set_axis_off()

ax[1].scatter(*embed.T, c=model.labels_)
ax[1].set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
def find_contours(image):
    # Find contours
    contours, _ = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    return contours

In [ ]:
def count_dots(contour):
    # Approximate the contour to a polygon
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.05 * peri, True)
    
    # If the polygon has 3 vertices, it's a triangle
    if len(approx) == 3:
        # Compute centroid of the contour
        M = cv2.moments(contour)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])

        return (cx, cy), approx

    return None, None

In [ ]:
def preprocess_image(image):
    # clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(30, 30))
    # image = clahe.apply(image)

    # Apply Gaussian blur to remove noise
    image = cv2.GaussianBlur(image, (5, 5), 0)
    image = cv2.medianBlur(image, 15)

    se=cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    bg=cv2.morphologyEx(image, cv2.MORPH_DILATE, se)
    image=cv2.divide(image, bg, scale=255)

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    #show(thresh)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, (3, 3), iterations=2)
    #thresh = cv2.erode(thresh, (3, 3), iterations=2)
    #thresh = cv2.dilate(thresh, (5, 5), iterations=2)
    show(thresh)

    return thresh

In [ ]:
image = cv2.imread('./images/Pict_2_1.bmp')

# Preprocess the image
output_image = None
for ch in cv2.split(image):
    if output_image is None:
        output_image = np.zeros_like(ch)
    processed_image = preprocess_image(ch)
    output_image = cv2.bitwise_or(output_image, processed_image)
#output_image = cv2.morphologyEx(output_image, cv2.MORPH_OPEN, (3, 3), iterations=5)
#output_image = cv2.erode(output_image, (3, 3), iterations=1)
show(output_image)
# Find contours
contours = find_contours(output_image.copy())

output_mask = np.zeros_like(image)
cv2.drawContours(output_mask, contours, -1, (0, 255, 0), 2)
show(output_mask)

# Create a copy of the original image to draw results
output_mask = np.zeros_like(image)
output_image = image.copy()

# List to store centroids and dot counts
centroids_validated = []
contours_validated = []

for contour in contours:
    dot_coords, approx = count_dots(contour)
    if dot_coords:
        centroids_validated.append(dot_coords)
        contours_validated.append(approx)
cv2.drawContours(output_image, contours_validated, -1, (0, 255, 0), 2)
cv2.drawContours(output_mask, contours_validated, -1, (0, 255, 0), 2)
show(output_mask)
show(output_image)

In [ ]:
contour = contours

output_image = image.copy()

# List to store centroids and dot counts
centroids_validated = []
contours_validated = []

for contour in contours:
    dot_coords, approx = count_dots(contour)
    if dot_coords:
        centroids_validated.append(dot_coords)
        contours_validated.append(approx)
cv2.drawContours(output_image, contours_validated, -1, (0, 255, 0), 2)
show(output_image)

In [ ]:
# Preprocess the image
output_image = preprocess_image(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
#output_image = cv2.morphologyEx(output_image, cv2.MORPH_OPEN, (3, 3), iterations=1)
#output_image = cv2.dilate(output_image, (15, 15), iterations=8)
show(output_image)
# Find contours
contours = find_contours(output_image.copy())

output_mask = np.zeros_like(image)
cv2.drawContours(output_mask, contours, -1, (0, 255, 0), 2)
show(output_mask)

# Create a copy of the original image to draw results
output_mask = np.zeros_like(image)
output_image = image.copy()

# List to store centroids and dot counts
centroids_validated = []
contours_validated = []

for contour in contours:
    dot_coords, approx = count_dots(contour)
    if dot_coords:
        centroids_validated.append(dot_coords)
        contours_validated.append(approx)
cv2.drawContours(output_image, contours_validated, -1, (0, 255, 0), 2)
cv2.drawContours(output_mask, contours_validated, -1, (0, 255, 0), 2)
show(output_mask)
show(output_image)

In [ ]:
image_orig = cv2.imread('./images/Pict_3_2.bmp')
image = cv2.cvtColor(image_orig, cv2.COLOR_BGR2GRAY)
image = cv2.GaussianBlur(image, (5, 5), 0)
image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))

edges = cv2.Canny(image, 50, 100)
show(edges)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# create a blank mask to store the segmented objects
mask = image_orig.copy()
fill_color = (0, 255, 0)

cv2.drawContours(mask, contours, -1, fill_color, 2)
show(mask)

In [ ]:
output_image = None
for ch in cv2.split(image):
    if output_image is None:
        output_image = np.zeros_like(ch)
    processed_image = preprocess_image(ch)
    output_image = cv2.bitwise_or(output_image, processed_image)
show(output_image)

In [ ]:
def set_default_color(image, target_color, threshold, default_color):
    
    # Define lower and upper bounds for the target color in LAB color space
    lower_bound = (target_color - threshold).astype(np.uint8)
    upper_bound = (target_color + threshold).astype(np.uint8)
    
    # Create a mask for pixels within the target color range
    mask = cv2.inRange(image, lower_bound, upper_bound)

    # Invert the mask
    mask = cv2.bitwise_not(mask)

    # Set the color of pixels outside the target color range to default color
    image[mask == 255] = default_color
    
    return image

target_color = np.array((107, 69, 48))
threshold = np.array((30, 10, 10))
default_color = np.array((255, 255, 255))
show(set_default_color(image.copy(), target_color, threshold, default_color))

In [ ]:
# Split image into channels
b, g, r = cv2.split(image_orig)

# Compute absolute differences for each channel
diff_b = cv2.absdiff(b, cv2.medianBlur(b, 5))
diff_g = cv2.absdiff(g, cv2.medianBlur(g, 5))
diff_r = cv2.absdiff(r, cv2.medianBlur(r, 5))

# Combine differences
diff = cv2.bitwise_or(cv2.bitwise_or(diff_b, diff_g), diff_r)

# Threshold the combined difference image
_, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_OTSU)
show(thresh)